In [1]:
import glob
import json
import numpy as np
import os
import shutil
from tqdm import tqdm
#from pycocotools.coco import COCO  # https://github.com/cocodataset/cocoapi/blob/master/PythonAPI/pycocoDemo.ipynb

In [2]:
percent_kept = 30

In [3]:
unfiltered_data_folder = "/media/data/dad/cnet/experiments/coco10novel/mix_n2000_o1_s1_p640"
unfiltered_annotation_path = os.path.join(unfiltered_data_folder, "annotation.json")
unfiltered_image_folder = os.path.join(unfiltered_data_folder, "images")
target_folder = unfiltered_data_folder + "_pfa_csl%d"%percent_kept

if not os.path.exists(target_folder):
    os.mkdir(target_folder)

target_image_folder = os.path.join(target_folder, "images")
target_annotation_path = os.path.join(target_folder, "annotation.json")

if not os.path.exists(target_image_folder):
    os.mkdir(target_image_folder)

In [4]:
with open(unfiltered_annotation_path, "r") as f:
    unfiltered_annotation = json.load(f)
#coco = COCO(unfiltered_annotation_path)

In [5]:
print(unfiltered_annotation.keys())
print(unfiltered_annotation["images"][0])
print(unfiltered_annotation["annotations"][0])
print(unfiltered_annotation["categories"][0])
print(unfiltered_annotation["images"][0].keys())
print(unfiltered_annotation["annotations"][0].keys())
print(unfiltered_annotation["categories"][0].keys())
print(len(unfiltered_annotation["images"]))
print(len(unfiltered_annotation["annotations"]))
print(len(unfiltered_annotation["categories"]))

dict_keys(['images', 'annotations', 'categories'])
{'license': 2, 'file_name': '000000568790.jpg', 'coco_url': 'http://images.cocodataset.org/train2017/000000568790.jpg', 'height': 426, 'width': 640, 'date_captured': '2013-11-21 00:25:07', 'flickr_url': 'http://farm2.staticflickr.com/1082/5110147496_81f18ceee0_z.jpg', 'id': 568790}
{'segmentation': [[290.06, 195.29, 297.72, 190.5, 304.42, 188.59, 315.91, 184.76, 323.57, 184.76, 329.31, 180.93, 336.97, 175.19, 341.76, 172.31, 345.59, 172.31, 349.42, 187.63, 355.16, 200.08, 369.52, 213.48, 378.13, 216.35, 384.84, 218.27, 389.62, 218.27, 399.2, 216.35, 406.85, 200.08, 411.64, 175.19, 411.64, 164.66, 409.73, 153.17, 403.02, 144.55, 392.49, 135.94, 388.67, 134.02, 384.84, 134.02, 381.96, 128.28, 375.26, 118.71, 371.43, 116.79, 362.82, 116.79, 355.16, 120.62, 352.29, 120.62, 341.76, 115.83, 330.27, 115.83, 312.08, 110.09, 300.59, 108.18, 301.55, 115.83, 314.95, 119.66, 329.31, 123.49, 347.5, 133.07, 357.07, 136.89, 361.86, 145.51, 364.73, 15

In [6]:
from collections import defaultdict

cat_ids = [cat['id'] for cat in unfiltered_annotation["categories"]]
print(cat_ids)

clip_scores = defaultdict(list)
for ann in unfiltered_annotation["annotations"]:
    clip_scores[ann['category_id']].append(ann["csl"])

csl_thres = {category_id:np.percentile(scores, 100 - percent_kept) for category_id, scores in clip_scores.items()}
print(len(clip_scores))
print(csl_thres)

[1, 2, 3, 4, 5, 6, 7, 9, 16, 17, 18, 19, 20, 21, 44, 62, 63, 64, 67, 72]
20
{1: 0.1980224609375, 18: 0.21834716796875, 20: 0.256201171875, 4: 0.2329345703125, 63: 0.2563720703125, 2: 0.24083251953125, 7: 0.240185546875, 17: 0.22265625, 3: 0.248876953125, 62: 0.24163818359375, 19: 0.2344970703125, 21: 0.26953125, 5: 0.19545898437499998, 6: 0.2467529296875, 64: 0.231689453125, 9: 0.2382568359375, 16: 0.22332763671875, 67: 0.183203125, 44: 0.21892089843749998, 72: 0.242236328125}


In [7]:
np.percentile(range(10), 20)

1.8

In [8]:
old_images = unfiltered_annotation["images"]
old_annos = unfiltered_annotation["annotations"]
new_annos = []
new_images = []

valid_image_id = set()
for anno in old_annos:
    if "segmentation" in anno or anno["csl"] > csl_thres[anno["category_id"]]:  # if segmentation in ann it's gt TODO: add sync flag
        new_annos.append(anno)
        valid_image_id.add(anno["image_id"])
for image in old_images:
    if image["id"] in valid_image_id:
        new_images.append(image)

unfiltered_annotation["images"] = new_images
unfiltered_annotation["annotations"] = new_annos

In [9]:
with open(target_annotation_path, "w+") as f:
    json.dump(unfiltered_annotation, f)

In [10]:
img_names = [img['file_name'] for img in unfiltered_annotation["images"]]
for img_name in img_names:
    shutil.copy(os.path.join(unfiltered_image_folder, img_name), os.path.join(target_image_folder, img_name))

In [11]:
print(len(new_images))
print(len(new_annos))
print(len(old_images))
print(len(old_annos))

729
812
2117
2200
